In [ ]:
"""
🎯 학습 목표: LCEL(LangChain Expression Language)을 사용한 복합 체인 구성 마스터
📚 사용된 LangChain 개념: ChatOpenAI, ChatPromptTemplate, LCEL 파이프 연산자, 체인 연결
🚀 실행 결과: 프로그래밍 언어에 대한 시 생성 → 시 분석 설명의 자동화된 워크플로우

LCEL을 사용한 두 단계 체인 구성 예제:
1단계: 프로그래밍 언어의 특징을 시로 표현하는 창의적 생성
2단계: 생성된 시를 분석하여 기술적 특징을 설명하는 해석적 분석
"""

# 🔧 필수 모듈 임포트
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# === 1단계: ChatOpenAI 모델 초기화 ===
# 🧠 개념: ChatOpenAI는 OpenAI GPT 모델을 LangChain에서 사용할 수 있게 해주는 래퍼 클래스
# 📌 핵심 설정값들의 의미:
#   - temperature=0.1: 낮은 값으로 일관성 있는 출력 보장 (창의적 작업에서도 품질 일관성)
#   - streaming=True: 실시간 응답 스트리밍으로 사용자 경험 향상
#   - callbacks: 응답 과정을 실시간으로 모니터링하고 출력
chat = ChatOpenAI(
    temperature=0.1,  # 📌 용도: 응답 일관성 제어, 타입: float, 창의성과 일관성의 균형점
    streaming=True,   # 📌 용도: 실시간 스트리밍 활성화, 타입: bool
    callbacks=[
        StreamingStdOutCallbackHandler(),  # 📌 용도: 콘솔 실시간 출력, 타입: CallbackHandler
    ],
)

# 💡 실무 팁: temperature 설정 가이드라인
# - 창의적 작업 (시, 소설): 0.7-0.9 (높은 창의성)
# - 기술 문서, 분석: 0.1-0.3 (높은 일관성)
# - 대화형 AI: 0.3-0.6 (균형)

# === 2단계: 시 생성용 프롬프트 템플릿 구성 ===
# 🧠 개념: ChatPromptTemplate.from_messages()는 역할별 메시지를 구조화하는 핵심 메서드
# 🔧 구조 분석:
#   - system 메시지: AI의 역할과 행동 방식 정의 (가장 중요!)
#   - human 메시지: 사용자 입력 템플릿, {language} 변수로 동적 입력 처리
poet_prompt = ChatPromptTemplate.from_messages([
    (
        "system",  # 📌 역할: 시스템 지시로 AI 페르소나 정의
        "당신은 프로그래밍 언어에 대한 시를 전문적으로 쓰는 시인입니다. 주어진 프로그래밍 언어의 본질과 특징을 담은 아름답고 창의적인 시를 작성해주세요.",
    ),
    ("human", "{language} 프로그래밍 언어에 대한 시를 써주세요."),  # 📌 변수: {language}를 통한 동적 입력
])

# 💡 실무 팁: system 메시지는 AI 행동의 80%를 결정함
# - 구체적 역할 정의: "전문 시인", "기술 해석가" 등 명확한 정체성 부여
# - 출력 품질 기준: "아름답고 창의적인" 등 품질 기대치 명시
# - 작업 범위 한정: 특정 도메인(프로그래밍 언어)으로 초점 집중

# === 3단계: 시 해석용 프롬프트 템플릿 구성 ===
# 🧠 개념: 두 번째 체인은 첫 번째 체인의 출력을 입력으로 받는 종속적 관계
# 📌 주목할 점: {poem} 변수는 이전 체인의 출력이 자동으로 매핑됨
explainer_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "당신은 프로그래밍 언어에 대한 시를 해석하는 전문가입니다. 주어진 시를 분석하여 프로그래밍 언어의 특징, 역사, 중요성과 어떻게 연관되어 있는지 명확하게 설명해주세요.",
    ),
    ("human", "이 프로그래밍 언어에 대한 시를 설명해주세요: {poem}"),  # 📌 변수: {poem}으로 이전 단계 결과 수신
])

# === 4단계: LCEL을 사용한 체인 구성 ===
# 🧠 개념: LCEL(LangChain Expression Language)은 파이프(|) 연산자로 컴포넌트 연결
# 🔧 체인 구성 단계별 분석:

# 4-1. 시 생성 체인: 프롬프트 → LLM
poet_chain = poet_prompt | chat  # 📌 흐름: 언어명 입력 → 시 생성

# 4-2. 시 설명 체인: 프롬프트 → LLM  
explainer_chain = explainer_prompt | chat  # 📌 흐름: 시 입력 → 시 분석

# 4-3. 최종 복합 체인: 두 체인의 연결
# 🧠 핵심 개념: {"poem": poet_chain}는 RunnableMap 생성
# 📊 데이터 흐름: input → poet_chain → {"poem": 시_결과} → explainer_chain → 최종_분석
final_chain = {"poem": poet_chain} | explainer_chain

# 💡 실무 팁: LCEL 체인 연결 패턴
# - 단순 연결: chain1 | chain2 (순차 실행)
# - 병렬 실행: {"result1": chain1, "result2": chain2} (동시 실행)
# - 조건부 실행: 복잡한 로직은 RunnableBranch 사용

# ⚠️ 주의사항: 
# - 변수명 일치: explainer_prompt의 {poem}과 {"poem": poet_chain}의 키 일치 필수
# - 스트리밍: 중간 체인 결과는 스트리밍되지 않음, 최종 결과만 스트리밍
# - 에러 처리: 중간 체인 실패 시 전체 체인 중단

# 🔗 관련 학습: 
# - 더 복잡한 체인 연결: Chapter_3_LCEL/3.4_Chaining_Chains.md
# - 프롬프트 엔지니어링: Chapter_4_Prompt_Engineering/4.1_FewShotPromptTemplate.md

In [ ]:
"""
🚀 실전 실행: 구성된 체인을 사용한 실제 워크플로우 테스트
💡 핵심 포인트: invoke() 메서드로 체인 실행, 딕셔너리 형태로 변수 전달
"""

# === 체인 실행 및 결과 분석 ===
# 🔧 실행 방법: final_chain.invoke()로 전체 워크플로우 실행
# 📌 입력 형식: {"language": "언어명"} 딕셔너리 형태로 변수 전달
# 📊 처리 과정: Java 입력 → 시 생성 → 시 분석 → 최종 결과 반환

# 🧠 개념 설명: invoke() 메서드의 동작 원리
# 1단계: {"language": "Java"}가 poet_chain으로 전달
# 2단계: poet_chain에서 Java에 대한 시 생성
# 3단계: 생성된 시가 {"poem": 시_내용} 형태로 explainer_chain에 전달  
# 4단계: explainer_chain에서 시 분석 및 설명 생성
# 5단계: 최종 분석 결과를 AIMessage 객체로 반환

result = final_chain.invoke({"language": "Java"})  # 📌 변수: language 키에 Java 값 전달

# 📊 결과 분석: 
print("📋 반환 타입:", type(result))  # AIMessage 또는 AIMessageChunk
print("📏 응답 길이:", len(result.content), "자")

# 💡 실무 팁: 다양한 프로그래밍 언어로 테스트
# languages_to_test = ["Python", "JavaScript", "Rust", "Go", "C++"]
# for lang in languages_to_test:
#     print(f"\n=== {lang} 분석 결과 ===")
#     result = final_chain.invoke({"language": lang})
#     print(result.content[:200] + "...")  # 처음 200자만 출력

# ⚠️ 주의사항:
# - invoke()는 동기 실행: 전체 워크플로우 완료까지 대기
# - 스트리밍: streaming=True 설정으로 실시간 출력 확인 가능
# - 에러 처리: 네트워크 오류, API 키 문제 등에 대한 예외 처리 필요

# 🚀 확장 아이디어:
# - 비동기 실행: ainvoke() 사용으로 성능 향상
# - 배치 처리: batch() 메서드로 여러 언어 동시 처리
# - 스트리밍 응답: stream() 메서드로 실시간 응답 처리
# - 결과 저장: 생성된 시와 분석을 파일로 저장하는 기능 추가

# 🔗 다음 학습 단계:
# - 체인 최적화: 캐싱, 병렬 처리로 성능 향상
# - 에러 처리: try-catch로 안정성 확보  
# - 사용자 인터페이스: 웹 또는 CLI 인터페이스 구축